In [1]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
DATABASE_URL = os.getenv("DATABASE_URL")
FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

In [3]:
import numpy as np
from langchain_core.tools import tool
from datetime import date, datetime
from typing import Optional, List
from langchain_core.runnables import RunnableConfig

In [4]:
from pydantic import BaseModel, Field
from typing import List

class Item(BaseModel):
    """
    Represents an item in an order with its details.
    """

    # order_id: str = Field(..., description="Unique identifier for the order.")
    item_id: int = Field(..., description="Unique identifier for the item.")
    quantity: float = Field(..., description="Quantity of the item ordered.")
    total_price: float = Field(..., description="Total price for the item.")

class Items(BaseModel):
    """
    Represents a collection of items in an order.
    """

    items: List[Item] = Field(..., description="List of items in the order.")


In [ ]:
@tool
def fetch_menu(config: RunnableConfig) -> list[dict]:
    """
    Fetches the pizza menu from the database and returns the items as a list of dictionaries.

    This function retrieves all the menu items from the 'menu' table, including details 
    like category and item name. It requires the customer's name to be provided in 
    the configuration. The function returns a list of dictionaries, where each dictionary 
    represents a menu item with its associated details.

    Args:
        config (RunnableConfig): Configuration object containing customer details.

    Returns:
        list[dict]: A list of dictionaries, each containing details of a menu item (e.g., category, item name).
    """
    
    configuration = config.get("configurable", {})
    customer_name = configuration.get("customer_name", None)
    if not customer_name:
        raise ValueError("No passenger ID configured.")

    conn = psycopg2.connect(DATABASE_URL)
    cursor = conn.cursor()

    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()

        # Fetch category and item_name
        cursor.execute("SELECT * FROM menu;")
        rows = cursor.fetchall()
        rows = [row for row in rows]
        column_names = [column[0] for column in cursor.description]
        results = [dict(zip(column_names, row)) for row in rows]
    except Exception as e:
        print("An error occurred:", e)

    finally:
        if conn:
            conn.close()
        if cursor:
            cursor.close()
    return results

In [6]:
fetch_menu.invoke(input={}, config={"configurable": {"customer_name": "An"}})

[{'item_id': 201,
  'category': 'Classic Pizzas',
  'item_name': 'Margherita',
  'description': 'Classic pizza with mozzarella and fresh basil',
  'size_options': '{Small,Medium,Large}',
  'price_small': Decimal('8.00'),
  'price_medium': Decimal('10.00'),
  'price_large': Decimal('14.00')},
 {'item_id': 202,
  'category': 'Classic Pizzas',
  'item_name': 'Pepperoni',
  'description': 'Loaded with pepperoni slices and melted cheese',
  'size_options': '{Small,Medium,Large}',
  'price_small': Decimal('10.00'),
  'price_medium': Decimal('12.00'),
  'price_large': Decimal('16.00')},
 {'item_id': 203,
  'category': 'Classic Pizzas',
  'item_name': 'Veggie Supreme',
  'description': 'Topped with bell peppers, onions, olives, and mushrooms',
  'size_options': '{Small,Medium,Large}',
  'price_small': Decimal('9.00'),
  'price_medium': Decimal('11.00'),
  'price_large': Decimal('15.00')},
 {'item_id': 204,
  'category': 'Classic Pizzas',
  'item_name': 'Hawaiian',
  'description': 'Ham and pin

In [ ]:
@tool
def order_food(items:Items, config: RunnableConfig) -> list[dict]:
    """
    Processes the customer's food order and inserts the order details into the database.

    This function creates a new order in the database for the specified customer, 
    generating an order ID. It then inserts the items ordered (with their quantity 
    and total price) into the 'order_items' table. The customer's name must be 
    provided in the configuration. The function returns the generated order ID.

    Args:
        items (Items): An object containing a list of items with their details 
                       (item_id, quantity, total_price).
        config (RunnableConfig): Configuration object containing customer details.

    Returns:
        int: The generated order ID.
    """
    
    configuration = config.get("configurable", {})
    customer_name = configuration.get("customer_name", None)
    if not customer_name:
        raise ValueError("No passenger ID configured.")

    conn = psycopg2.connect(DATABASE_URL)
    cursor = conn.cursor()

    try:
        query = """INSERT INTO orders (customer_name) VALUES (%s) RETURNING order_id"""
        cursor.execute(query, (customer_name,))
        order_id = cursor.fetchone()[0]
        items = items.items
        items = [(order_id,) + (item.item_id, item.quantity, item.total_price) for item in items]
        query = """INSERT INTO order_items (order_id, item_id, quantity, total_price) VALUES (%s, %s, %s, %s)"""
        cursor.executemany(query, items)
        conn.commit()
    except Exception as e:
        print("An error occurred:", e)

    finally:
        if conn:
            conn.close()
        if cursor:
            cursor.close()
    return order_id

In [8]:
order_items = [
    Item(item_id=201, quantity=1, total_price=10.00),  # BBQ Chicken
    Item(item_id=202, quantity=2, total_price=20.00)   # Pepperoni
]



order_food.invoke(input={"items": Items(items=order_items)}, config={"configurable": {"customer_name": "An"}})

1

In [ ]:
@tool
def fetch_user_order_information(config: RunnableConfig) -> list[dict]:
    """
    Fetches the most recent order details for a specific customer from the database.
    
    This function retrieves the order ID, customer name, order date, item ID, 
    quantity, and total price for the latest order placed by the specified customer. 
    The customer's name must be provided in the configuration. The function returns 
    a list of dictionaries containing the order information.

    Returns:
        list[dict]: A list of dictionaries with order details, including item IDs, 
                    quantities, and total prices.
    """

    configuration = config.get("configurable", {})
    customer_name = configuration.get("customer_name", None)
    if not customer_name:
        raise ValueError("No passenger ID configured.")

    try:
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()

        query = """
        SELECT orders.order_id, orders.customer_name, orders.order_date, order_items.item_id, order_items.quantity, order_items.total_price
        FROM orders
        JOIN order_items ON orders.order_id = order_items.order_id
        WHERE orders.order_date = (SELECT MAX(orders.order_date) FROM orders) AND orders.customer_name = %s
        """
        cursor.execute(query, (customer_name,))
        rows = cursor.fetchall()
        rows = [row for row in rows]
        column_names = [column[0] for column in cursor.description]
        results = [dict(zip(column_names, row)) for row in rows]
    except Exception as e:
        print("An error occurred:", e)

    finally:
        if conn:
            conn.close()
        if cursor:
            cursor.close()
    return results

In [10]:
fetch_user_order_information.invoke(input={}, config={"configurable": {"customer_name": "An"}})

[{'order_id': 1,
  'customer_name': 'An',
  'order_date': datetime.datetime(2024, 12, 7, 15, 16, 36, 582094),
  'item_id': 201,
  'quantity': 1,
  'total_price': Decimal('10.00')},
 {'order_id': 1,
  'customer_name': 'An',
  'order_date': datetime.datetime(2024, 12, 7, 15, 16, 36, 582094),
  'item_id': 202,
  'quantity': 2,
  'total_price': Decimal('20.00')}]

In [11]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode


def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

In [12]:
from typing import Annotated


from langchain_core.runnables import Runnable, RunnableConfig
from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_info: str


class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            configuration = config.get("configurable", {})
            passenger_id = configuration.get("customer_name", None)
            state = {**state, "user_info": passenger_id}
            result = self.runnable.invoke(state)
            # If the LLM happens to return an empty response, we will re-prompt it
            # for an actual response.
            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_together import ChatTogether

assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful customer support assistant for Pizza Innovation "
            " Use the provided tools to search for order information, order food to assist the user's queries. "
            " When searching, be persistent. Expand your query bounds if the first search returns no results. "
            " If a search comes up empty, expand your search before giving up."
            "\n\nCurrent user:\n<User>\n{user_info}\n</User>"
            "\nCurrent time: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful customer support assistant for Pizza Innovation"
            "Use the provided tools to assist the user's queries. "
            "When searching, be persistent. Expand your query bounds if the first search returns no results. "
            "If a search comes up empty, expand your search before giving up."
            "\nCurrent time: {time}.",
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)

llm = ChatTogether(model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo", max_retries=3, temperature=0)
# llm = ChatFireworks(model="accounts/fireworks/models/llama-v3p1-8b-instruct", max_retries=3)
tools = [fetch_menu, fetch_user_order_information, order_food]

llm_with_tools = llm.bind_tools(tools)
assistant_runnable = assistant_prompt | llm_with_tools

In [24]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

builder = StateGraph(State)


# Define nodes: these do the work
builder.add_node("assistant", Assistant(assistant_runnable))
builder.add_node("tools", create_tool_node_with_fallback(tools))
# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

# The checkpointer lets the graph persist its state
# this is a complete memory for the entire graph.
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [25]:
import uuid

thread_id = str(uuid.uuid4())

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "thread_id": thread_id,
        "customer_name": "Long",
    }
}

In [26]:
_printed = set()

questions = ["I want to check menu.",
             "I want to order Pepperoni medium and BBQ Chicken small",
             "I want to check my order."]

for ques in questions:
    events = graph.stream(
            {"messages": ("human", ques)}, config, stream_mode="values"
        )
    for event in events:
        _print_event(event, _printed)

================================ Human Message =================================

I want to check menu.
================================== Ai Message ==================================
Tool Calls:
  fetch_menu (call_a84q8gojeopf8kzpqio7zpgh)
 Call ID: call_a84q8gojeopf8kzpqio7zpgh
  Args:
================================= Tool Message =================================
Name: fetch_menu

[{'item_id': 201, 'category': 'Classic Pizzas', 'item_name': 'Margherita', 'description': 'Classic pizza with mozzarella and fresh basil', 'size_options': '{Small,Medium,Large}', 'price_small': Decimal('8.00'), 'price_medium': Decimal('10.00'), 'price_large': Decimal('14.00')}, {'item_id': 202, 'category': 'Classic Pizzas', 'item_name': 'Pepperoni', 'description': 'Loaded with pepperoni slices and melted cheese', 'size_options': '{Small,Medium,Large}', 'price_small': Decimal('10.00'), 'price_medium': Decimal('12.00'), 'price_large': Decimal('16.00')}, {'item_id': 203, 'category': 'Classic Pizzas', 'item_